In [ ]:
import casadi as cs
import l4casadi as l4c


naive_mlp = l4c.naive.MultiLayerPerceptron(2, 128, 1, 2, 'Tanh')
l4c_model = l4c.L4CasADi(naive_mlp, model_expects_batch_dim=True)

x_sym = cs.MX.sym('x', 2, 1)
y_sym = l4c_model(x_sym)
f = cs.Function('y', [x_sym], [y_sym])
df = cs.Function('dy', [x_sym], [cs.jacobian(y_sym, x_sym)])
ddf = cs.Function('ddy', [x_sym], [cs.hessian(y_sym, x_sym)[0]])

x = cs.DM([[0.], [2.]])
print(l4c_model(x))
print(f(x))
print(df(x))
print(ddf(x))

In [ ]:
import tinycudann as tcnn

mlp = tcnn.Network(
    n_input_dims=2,
    n_output_dims=1,
    network_config={
        "otype": "CutlassMLP",
        "activation": "ReLU",
        "output_activation": "None",
        "n_neurons": 256,
        "n_hidden_layers": 1,
    },
)

l4c_mlp = l4c.L4CasADi(mlp)

# Declare variables
x = cs.MX.sym("x", 2, 1)
y = l4c_mlp(x)

In [ ]:
import casadi as cs
import torch.nn

import l4casadi as l4c

# Declare variables
x = cs.MX.sym("x", 2)


# Form the NLP
class PyTorchObjectiveModel(torch.nn.Module):
    def forward(self, input):
        return torch.square(input[0]) + torch.square(input[1])[..., None]


f = PyTorchObjectiveModel()  # objective
f = l4c.L4CasADi(f, name='f', model_expects_batch_dim=False)(x)


class PyTorchConstraintModel(torch.nn.Module):
    def forward(self, input):
        return (input[0] + input[1] - 10)[..., None]


g = PyTorchConstraintModel()  # constraint
g = l4c.L4CasADi(g, name='g', model_expects_batch_dim=False)(x)

nlp = {'x': x, 'f': f, 'g': g}


# Pick an NLP solver
MySolver = "ipopt"
# MySolver = "worhp"
# MySolver = "sqpmethod"

# Solver options
opts = {}
if MySolver == "sqpmethod":
    opts["qpsol"] = "qpoases"
    opts["qpsol_options"] = {"printLevel": "none"}  # type: ignore[assignment]

# Allocate a solver
solver = cs.nlpsol("solver", MySolver, nlp, opts)

# Solve the NLP
sol = solver(lbg=0)

# Print solution
print("-----")
print("objective at solution = ", sol["f"])
print("primal solution = ", sol["x"])
print("dual solution (x) = ", sol["lam_x"])
print("dual solution (g) = ", sol["lam_g"])